In [1]:
import os, sys
# import mcvine modules
from instrument.geometry.pml import weave
from instrument.geometry import  operations

In [2]:
thisdir = os.path.abspath(os.path.dirname("__file__"))
libpath = os.path.join(thisdir, '../c3dp_source')
if not libpath in sys.path:
    sys.path.insert(0, libpath)

In [3]:
import collimator_support as colli_original

In [4]:
import SCADGen.Parser
from create_collimator_geometry import Collimator_geom
from c3dp.instrument.cellgeometry.clampcell_geo import Clampcell
from c3dp.instrument.cellgeometry.DAC_geo import DAC

In [5]:
clampcell=Clampcell(total_height=True)
outer_body=clampcell.outer_body()
inner_sleeve=clampcell.inner_sleeve()
sample=clampcell.sample()

cell_sample_assembly=operations.unite(operations.unite(outer_body, sample), inner_sleeve)

In [7]:
scad_flag = True  ########CHANGE CAD FLAG HERE

if scad_flag is True:
    samplepath = os.path.join(thisdir, '../figures')
else:
    samplepath = os.path.join(thisdir, '../sample')

In [9]:
for coll_length in [195.]: #100, 230,380


    channel_length=195. #32, 17

    min_channel_wall_thickness=1.

#     coll = colli_original.Collimator_geom()
    coll = Collimator_geom()
    coll.set_constraints(max_coll_height_detector=230., max_coll_width_detector=230.,
                        min_channel_wall_thickness=min_channel_wall_thickness,
                        max_coll_length=coll_length, min_channel_size=3.,
                         truss_base_thickness=30., trass_final_height_factor=0.45,
                         touch_to_halfcircle=6 ,SNAP_acceptance_angle=False)


    coll.set_parameters(number_channels=3.,channel_length =channel_length)

    filename = 'coll_geometry_{coll_length}_{coll_height}_{coll_width}_{channel_length}_{wall_thickness}.xml'.\
        format(coll_length=coll_length, coll_height=coll.max_coll_height_detector, coll_width=coll.max_coll_height_detector, channel_length=channel_length, wall_thickness=min_channel_wall_thickness)

    outputfile = os.path.join(samplepath, filename)


    supports=coll.support()

    truss=coll.support_design()

    coli = coll.gen_one_col(collimator_Nosupport=False)

#     collimator=coll.gen_collimators_xml(multiple_collimator=False, scad_flag=scad_flag,detector_angles=[0], collimator_Nosupport=False, coll_file=outputfile)



both=operations.unite(cell_sample_assembly, coli)
# both= truss
# both=supports
file='colli_anvil'
filename='%s.xml'%(file)
outputfile=os.path.join(samplepath, filename)
with open (outputfile,'wt') as file_h:
    weave(both,file_h, print_docs = False)


In [10]:
p = SCADGen.Parser.Parser(outputfile)
p.createSCAD()
test = p.rootelems[0]

In [11]:
cadFile_name='%s.scad'%(file)
cad_file_path=os.path.abspath(os.path.join(samplepath, cadFile_name))

In [12]:
cad_file_path

'/home/fi0/Collimator_3D/c3dp/figures/colli_anvil.scad'

In [13]:
!vglrun openscad {cad_file_path}

/bin/sh: 1: Syntax error: "(" unexpected


In [5]:
import cadquery as cq
from cq_jupyter import Assembly, Part, Edges, Faces, display, exportSTL

SyntaxError: only named arguments may follow *expression (geom.py, line 40)